<a href="https://colab.research.google.com/github/JoergNeumann/GenAI/blob/main/SentimentAnalyse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment-Analyse
In dieser Lektion werden Sie die Stimmung und Themen aus Produktbewertungen und Nachrichtenartikeln ableiten.

## Setup

In [ ]:
!pip install openai==1.55.3 httpx==0.27.2 --force-reinstall --quiet

In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
import os
import openai

# OpenAI API Key aus Colab Secret auslesen
from google.colab import userdata
openai.api_key = userdata.get('OPENAI_API_KEY')

In [2]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
    )
    return response.choices[0].message.content

## Produktbewertung

In [3]:
lamp_review = """
Ich brauchte eine schöne Lampe für mein Schlafzimmer, und diese hier hatte \
zusätzlichen Stauraum und keinen allzu hohen Preis. Die Lieferung war super \
schnell. Leider ist die Halterung der Lampe während des Transports kaputt gegangen, \
aber der Kundenservice des Unternehmens hat mir glücklicherweise und ohne Probleme \
eine neue geschickt. Diese kam auch innerhalb weniger Tage an. Der Zusammenbau war \
einfach. Mir fehlte zwar ein Teil, aber der Kundendienst hat mir das fehlende Teil \
sehr schnell besorgt! Lumina scheint mir ein tolles Unternehmen zu sein, das sich \
um seine Kunden und Produkte kümmert!
"""

## Sentiment erkennen (positiv/negativ)

In [4]:
prompt = f"""
Was ist die Gesamtaussage der folgenden Produktbewertung, die mit drei Backticks gekennzeichnet ist?

Review-Text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Die Gesamtaussage der Produktbewertung ist positiv. Der Kunde lobt die Lampe für ihr Design, den zusätzlichen Stauraum und den angemessenen Preis. Obwohl es Probleme mit der Halterung und einem fehlenden Teil gab, lobt der Kunde den schnellen und problemlosen Kundenservice des Unternehmens. Insgesamt scheint der Kunde mit dem Produkt und dem Unternehmen zufrieden zu sein.


In [5]:
prompt = f"""
Welches Sentiment vermittelt die folgende Produktbewertung, die mit drei Backticks gekennzeichnet ist?

Gib die Antwort in einem Wort, entweder "positiv" oder "negativ".

Review-Text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

positiv


## Emotionen erkennen

In [6]:
prompt = f"""
Identifiziere eine Liste von Emotionen, die der Verfasser der folgenden \
Bewertung ausdrückt. Beschränke dich dabei auf maximal fünf Begriffe. \
Formatiere deine Antwort als kommagetrennte Liste von kleingeschriebenen Wörtern.

Review-Text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

zufrieden, dankbar, erleichtert, beeindruckt, positiv


## Ärger erkennen

In [7]:
prompt = f"""
Ist der Verfasser der folgenden Bewertung wütend? Die Bewertung ist mit drei Backticks gekennzeichnet.
Gib deine Antwort mit Ja oder Nein.

Review-Text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

Nein.


## Produkte und Firmennamen aus Kundenbewertungen extrahieren

In [8]:
prompt = f"""
Finde im folgenden Bewertungstext folgende Informationen:
- Gekauftes Produkt
- Hersteller des Produkts
Der Text ist mit drei Backticks (```) abgegrenzt.

Gib die Antwort als JSON-Objekt mit den Schlüsseln 'Item' und 'Brand' \
zurück. Verwende für fehlende Informationen 'unbekannt'.

Die Antwort soll so kurz wie möglich sein.

Review-Text: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
    "Item": "Lampe",
    "Brand": "Lumina"
}


## Mehrere Aufgaben gleichzeitig erledigen

In [9]:
prompt = f"""
Identifizieren Sie die folgenden Punkte aus dem Rezensionstext:
- Stimmung (positiv oder negativ)
- Drückt der Rezensent Ärger aus? (wahr oder falsch)
- Vom Rezensenten gekauftes Produkt
- Firma, die das Produkt hergestellt hat

Die Rezension ist mit dreifachen Backticks begrenzt. \
Formatieren Sie Ihre Antwort als JSON-Objekt mit \
"Sentiment", "Anger", "Item" und "Brand" als Schlüssel.
Wenn die Information nicht vorhanden ist, verwenden Sie "unknown" \
als Wert.
Halten Sie Ihre Antwort so kurz wie möglich.
Formatieren Sie den Anger-Wert als booleschen Wert.

Rezensionstext: '''{lamp_review}'''
"""
response = get_completion(prompt)
print(response)

{
    "Sentiment": "positiv",
    "Anger": false,
    "Item": "unknown",
    "Brand": "Lumina"
}


## Themen identifizieren

In [10]:
story = """
Bei einer kürzlich von der Regierung durchgeführten Umfrage wurden Beschäftigte \
des öffentlichen Dienstes gebeten, ihre Zufriedenheit mit der Behörde, in der sie \
arbeiten, zu bewerten. Die Ergebnisse zeigten, dass die NASA mit einer Zufriedenheitsrate \
von 95 % die beliebteste Behörde war.
Ein NASA-Mitarbeiter, John Smith, kommentierte die Ergebnisse mit den Worten: "Es überrascht \
mich nicht, dass die NASA ganz oben steht. Es ist ein großartiger Arbeitsplatz mit tollen \
Kollegen und unglaublichen Möglichkeiten. Ich bin stolz darauf, Teil einer so innovativen Organisation zu sein."
Die Ergebnisse wurden auch vom NASA-Managementteam begrüßt. Direktor Tom Johnson erklärte: \
"Wir freuen uns sehr zu hören, dass unsere Mitarbeiter mit ihrer Arbeit bei der NASA \
zufrieden sind. Wir haben ein talentiertes und engagiertes Team, das unermüdlich an der \
Erreichung unserer Ziele arbeitet, und es ist fantastisch zu sehen, dass sich die harte Arbeit auszahlt."
Die Umfrage ergab außerdem, dass die Sozialversicherungsbehörde die niedrigste Zufriedenheitsrate \
aufweist. Nur 45 % der Mitarbeiter gaben an, mit ihrem Arbeitsplatz zufrieden zu sein. Die Regierung \
hat zugesagt, die von den Mitarbeitern in der Umfrage geäußerten Bedenken anzugehen und daran zu arbeiten, \
die Zufriedenheit mit dem Arbeitsplatz in allen Abteilungen zu verbessern.
"""

## 5 Themen identifizieren

In [11]:
prompt = f"""
Finde fünf Themen, die in folgendem Text diskutiert werden:
Der Text ist mit drei Backticks abgegrenzt.
Schreibe jedes Thema als ein oder zwei Wörter.

Formatiere deine Antwort als Kommagetrennte Liste.

Text-Beispiel: '''{story}'''
"""
response = get_completion(prompt)
print(response)

NASA, Mitarbeiterzufriedenheit, Sozialversicherungsbehörde, Regierungsumfrage, Arbeitsplatzzufriedenheit


In [12]:
response.split(sep=',')

['NASA',
 ' Mitarbeiterzufriedenheit',
 ' Sozialversicherungsbehörde',
 ' Regierungsumfrage',
 ' Arbeitsplatzzufriedenheit']